Installing dependencies

In [ ]:
!pip install torch
!pip install tensorflow
!pip install transformers
!pip install datasets 
!pip install transformers[sentencepiece]
!pip install sacrebleu
!pip install sentencepiece

Google Drive connection

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
# !cd gdrive/MyDrive/Colab\ Notebooks/MLGS_1; ls
# !cp gdrive/MyDrive/Colab\ Notebooks/MLGS_1/nf_utils.py .

Mounted at /content/gdrive


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
!cd gdrive/MyDrive/Colab\ Notebooks/GR; ls
!cp -r gdrive/MyDrive/Colab\ Notebooks/GR/data .

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
data  hsb-dsb-MT.ipynb	Huggingface_tutorial.ipynb


# Translation

## Preprocessing

In [ ]:
# from datasets import load_dataset, load_from_disk, save_to_disk
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import pipeline, AutoTokenizer

src_train_filename = "data/train_dsb_hsb.dsb"
target_train_filename = "data/train_dsb_hsb.hsb"
def get_translation_dataframe(data_dsb, data_hsb):
  return pd.DataFrame([
                       {'id': id, 
                        'translation': {'dsb': sentence_dsb, 'hsb': sentence_hsb}
                        } for id, sentence_dsb, sentence_hsb in zip(data_dsb.index, data_dsb[0], data_hsb[0])
                        ])

train_data_dsb = pd.read_csv(src_train_filename, sep='\r\t', header=None)
train_data_hsb = pd.read_csv(target_train_filename, sep='\r\t', header=None)
val_data_dsb = pd.read_csv("data/valid_dsb_hsb.dsb", sep='\r\t', header=None)
val_data_hsb = pd.read_csv("data/valid_dsb_hsb.hsb", sep='\r\t', header=None)
test_data_dsb = pd.read_csv("data/test_dsb_hsb.dsb", sep='\r\t', header=None)
test_data_hsb = pd.read_csv("data/test_dsb_hsb.hsb", sep='\r\t', header=None)

train_df = Dataset.from_pandas(get_translation_dataframe(train_data_dsb, train_data_hsb))
val_df = Dataset.from_pandas(get_translation_dataframe(val_data_dsb, val_data_hsb))
test_df = Dataset.from_pandas(get_translation_dataframe(test_data_dsb, test_data_hsb))

split_datasets = DatasetDict({"train": train_df, "validation": val_df, "test": test_df})


# [-1]["translation"]

# squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

# squad_it_dataset.save_to_disk("squad-it")

# squad_it_dataset_reloaded = load_from_disk("squad-it")

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


## Tokenizer

Instead, the labels corresponding to a padding token should be set to -100 so they are ignored in the loss computation.

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer
)
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import transformers, tokenizers
from transformers import PreTrainedTokenizerFast

src_lang = "dsb"
target_lang = "hsb"
max_input_length = 128
max_target_length = 128

def get_training_corpus(lang, dataset):
    for i in range(0, len(dataset), 1000):
        yield [translation[lang] for translation in split_datasets["train"][i : i + 1000]["translation"]]


# tokenizer = ByteLevelBPETokenizer()
# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
# tokenizer.train(files=src_train_filename, vocab_size=2_500, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("</s>", tokenizer.token_to_id("</s>")),
#     ("<s>", tokenizer.token_to_id("<s>")),
# )
# tokenizer.enable_truncation(max_length=512)
# tokenizer.save("hsb-BPE.tokenizer.json")

# Load it using transformers
# tokenizer = PreTrainedTokenizerFast(tokenizer_file="hsb-BPE.tokenizer.json")
def create_tokenizer(lang):
    tokenizer = Tokenizer(models.BPE())
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
    trainer = trainers.BpeTrainer(vocab_size=4000, special_tokens=["<|endoftext|>"])
    tokenizer.train_from_iterator(get_training_corpus(target_lang, train_df), trainer=trainer)

    sample_sentence_hsb = split_datasets["train"][0]["translation"][lang]
    encoding = tokenizer.encode(sample_sentence_hsb)
    print(encoding.tokens)
    tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
    tokenizer.decoder = decoders.ByteLevel()
    print(tokenizer.decode(encoding.ids))

    return PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        bos_token="<|endoftext|>",
        eos_token="<|endoftext|>",
    )

src_tokenizer = create_tokenizer(src_lang)
target_tokenizer = create_tokenizer(target_lang)

def preprocess_function(examples):
    inputs = [ex[src_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = src_tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with src_tokenizer.as_target_tokenizer():
        labels = target_tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)


['Å', '¹', 'in', 'sa', 'j', 'Å¡', 'na', 'Ġt', 'owa', 'ri', 'Å¡', 'nosÄĩ', ',', 'Ġtak', 'Ġjo', 'ĠpÃ³', 'd', 'Å¡', 'ma', 'r', 'nu', 'ÅĤ', ',', 'Ġtrjeba', 'Ġku', 'raÅ¾', 'u', 'Ġa', 'Ġza', 'g', 'roni', 'tosÄĩ', '.']
Źinsajšna towarišnosć, tak jo pódšmarnuł, trjeba kuražu a zagronitosć.
['DÅºen', 'sni', 'Å¡a', 'ĠtowarÅ¡', 'nosÄĩ', ',', 'Ġtak', 'ĠwÃ³n', 'ĠpodÅ¡mÃ³rny', ',', 'Ġtrjeba', 'Ġz', 'mu', 'Å¾i', 'tosÄĩ', 'Ġa', 'ĠzamoÅĤwitosÄĩ', '.']
Dźensniša towaršnosć, tak wón podšmórny, trjeba zmužitosć a zamołwitosć.


  0%|          | 0/53 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AutoTokenizer
from datasets import load_metric
import numpy as np

src_sentence = split_datasets["train"][0]["translation"][src_lang]

model_checkpoint = "Helsinki-NLP/opus-mt-cs-de"
# tokenizer_cs = AutoTokenizer.from_pretrained("xlnet-base-cased") # XLNetTokenizer.from_pretrained("xlnet-base-cased")
translator = pipeline("translation", model=model_checkpoint) #, tokenizer=src_tokenizer) #, tokenizer=tokenizer_cs)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(src_tokenizer, model=model)
# batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
# batch.keys()

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds, tokenizer):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
print(src_sentence)
print(translator(src_sentence))
print(translator("Ako se maš?"))

Źinsajšna towarišnosć, tak jo pódšmarnuł, trjeba kuražu a zagronitosć.
[{'translation_text': 'Insajšna towarišnosć, tak jo pódšmarnuł, trjeba kuržu a zagronitosć.'}]
[{'translation_text': "Wie geht's dir?"}]


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"bartek-baseline-hsb-to-dsb",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=src_tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()
trainer.evaluate(max_length=max_target_length)
# trainer.push_to_hub(tags="translation", commit_message="Training complete")


ValueError: ignored

# Part 1

Working with GIZA++

In [3]:
!cd gdrive/MyDrive/Colab\ Notebooks/GR/giza-py; pip install -r requirements.txt


     |████████████████████████████████| 191 kB 4.1 MB/s 
     |████████████████████████████████| 749 kB 40.2 MB/s 
     |████████████████████████████████| 178 kB 53.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: chardet
    Found existing installation: chardet 3.0.4
    Uninstalling chardet-3.0.4:
      Successfully uninstalled chardet-3.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires chardet<4,>=3.0.2, but you have chardet 4.0.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
!cd gdrive/MyDrive/Colab\ Notebooks/GR/boost_1_79_0; ./bootstrap.sh --prefix=./build --with-libraries=thread,system; ./b2 install


/bin/bash: ./bootstrap.sh: /bin/sh: bad interpreter: Permission denied
/bin/bash: ./b2: Permission denied
